In [1]:
import pandas as pd
import numpy as np
import json
    
df = pd.read_csv('../../data/imp_exp_prod_flat_all.csv')
df.drop(columns='Row Number', inplace=True)
df = df[['Area', 'Item', 'Year', 'Production', 'Import Quantity', 'Export Quantity']]
df.rename(columns={'Area': 'Country', 'Item':'Produce', 'Production':'Production Quantity'}, inplace=True)
df.head()

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
0,Armenia,Potatoes,2000,290260.0,390.0,300.0
1,Armenia,Potatoes,2001,363834.0,2290.0,0.0
2,Armenia,Potatoes,2002,374263.0,1918.0,0.0
3,Armenia,Potatoes,2003,507518.0,2853.0,7.0
4,Armenia,Potatoes,2004,576427.0,1553.0,0.0


In [2]:
# create a lookup table for countries and their centroid locations
lookup = pd.read_csv('../../data/country_centroids.csv')
lookup.rename(columns={'country':'code'}, inplace=True)
lookup.index = lookup['name']
(lookup.drop(columns=['name'])).to_csv('../../data/countries_lookup.csv')


# FIXME --------------------------------------------------------------------------
# ********************************************************************************
# Assuming that China is a sum of mainland, taiwan, hong kong and macao
# 'Serbia and Montenegro' - set coordinates to average of Serbia and Montenegro
# 'South Sudan' and 'Sudan (former)'- set coordinates to Sudan's coordinates

# print regions that are not present in the countries lookup table
set(df[~df['Country'].isin(lookup['name'])]['Country'])

{'Africa',
 'Americas',
 'Asia',
 'Australia & New Zealand',
 'Caribbean',
 'Central America',
 'Central Asia',
 'China',
 'Eastern Africa',
 'Eastern Asia',
 'Eastern Europe',
 'Europe',
 'European Union',
 'Land Locked Developing Countries',
 'Least Developed Countries',
 'Low Income Food Deficit Countries',
 'Melanesia',
 'Middle Africa',
 'Net Food Importing Developing Countries',
 'Northern Africa',
 'Northern America',
 'Northern Europe',
 'Occupied Palestinian Territory',
 'Oceania',
 'Polynesia',
 'Small Island Developing States',
 'South America',
 'South-Eastern Asia',
 'Southern Africa',
 'Southern Asia',
 'Southern Europe',
 'Western Africa',
 'Western Asia',
 'Western Europe',
 'World'}

In [3]:
# remove the regions printed above from the dataframe
df = df[df['Country'].isin(lookup.index)]

# randomly print 10 entries from the dataframe for an example of what it contains
df.sample(n=10)

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
71051,Venezuela (Bolivarian Republic of),Oranges,2007,389842.0,0.0,1765.0
90197,Cuba,Cotton lint,2001,0.0,1997.0,0.0
39852,Poland,"Fruit, fresh nes",2014,75220.0,7818.0,2098.0
46967,Bolivia (Plurinational State of),Cotton lint,2013,29000.0,544.0,248.0
43657,Romania,Apples,2007,475370.0,53897.0,19177.0
24544,Mexico,"Spices, nes",2001,2804.0,357.0,1415.0
39816,Poland,Blueberries,2012,11251.0,1342.0,6564.0
25001,Mongolia,"Nuts, nes",2007,4200.0,177.0,23.0
75450,Luxembourg,Cherries,2005,260.0,188.0,1.0
153344,Greece,Triticale,2007,0.0,37.0,0.0


In [4]:
# calculate top 10 producers, importers and exporters for each produce in each year
top10_statistics = {}
top10_statistics['produce'] = produceSet = sorted(set(df['Produce']))
top10_statistics['largest_quantity'] = int((df.nlargest(1, 'Production Quantity'))['Production Quantity'])

for produce in produceSet: 
    years = sorted(set(df[df['Produce'] == produce]['Year']))
    yearStatistics = {}
    yearStatistics['available_years'] = years
    individualYears = {}
    
    for year in years:
        selected = df[(df['Produce'] == produce) & (df['Year'] == year)]
        topProducers = selected.nlargest(10, 'Production Quantity')
        topImporters = selected.nlargest(10, 'Import Quantity')
        topExporters = selected.nlargest(10, 'Export Quantity')
        individualYears[year] = {
            'Producers': {
                'countries': list(topProducers['Country']),
                'production': list(topProducers['Production Quantity']),
                'import': list(topProducers['Import Quantity']),
                'export': list(topProducers['Export Quantity'])
            },
            'Importers': {
                'countries': list(topImporters['Country']),
                'production': list(topImporters['Production Quantity']),
                'import': list(topImporters['Import Quantity']),
                'export': list(topImporters['Export Quantity'])
            },
            'Exporters': {
                'countries': list(topExporters['Country']),
                'production': list(topExporters['Production Quantity']),
                'import': list(topExporters['Import Quantity']),
                'export': list(topExporters['Export Quantity'])
            }
        }
        
    yearStatistics['top10_per_year'] = individualYears
    top10_statistics[produce] = yearStatistics

In [5]:
# dump top 10 statistics to a json file
json.dump(top10_statistics, open('../../data/top10_imp_exp_prod.json', 'w'))